In [ ]:
from IPython.display import display, HTML, Math, Markdown
display(HTML("<style>.container { width:95% !important; }</style>"))

In [ ]:
import coralme
from coralme.builder.main import MEBuilder, MEReconstruction, METroubleshooter
from coralme.builder.curation import MECurator, MEManualCuration


In [ ]:
org = 'cljungdahlii'
organism = './{:s}/organism.json'.format(org)
inputs = './{:s}/input.json'.format(org)

### Full

In [ ]:
builder = MEBuilder(*[organism, inputs])

In [ ]:
builder.generate_files(overwrite=True)
builder.save_builder_info()

In [ ]:
builder.build_me_model(overwrite=False)

In [ ]:
builder.troubleshoot(growth_key_and_value = { builder.me_model.mu : 0.001 })

### Quick

In [ ]:
builder = MEBuilder(*['./{:s}/coralme-config.yaml'.format(org)])

In [ ]:
builder.build_me_model(overwrite=False, update=True, prune=False)

In [ ]:
builder.troubleshoot(growth_key_and_value = { builder.me_model.mu : 0.001 })

### Load builder

In [ ]:
builder = MEBuilder(*['./{:s}/coralme-config.yaml'.format(org)])
builder.me_model = builder.load(builder.configuration['out_directory']+'MEModel-step2-{}.pkl'.format(org))

In [ ]:
for r in builder.me_model.metabolites.query("trnaser_c")[0].reactions:
    print(r.id, r.reaction)

In [ ]:
builder.find_issue('')

### Curation notes

In [ ]:
builder.curation_notes

### Notes

In [ ]:
from coralme.builder.helper_functions import *
pandas.set_option('display.max_colwidth', None)

In [ ]:
import pandas as pd

In [ ]:
builder = MEBuilder(*['./{:s}/coralme-config.yaml'.format(org)])
builder.me_model = builder.load(builder.configuration['out_directory']+'MEModel-step2-{}.pkl'.format(org))
me = builder.me_model

In [ ]:
a = 0
for i in me.all_genes:
    if not i.functions:
        a += 1
        print(a)
        print(i.id)
        print(i.complexes)
        print(i.functions)
        print()

In [ ]:
for r in me.get('generic_Def').reactions:
    print(r.id)
    print(r.metabolites)
    print()
    break

In [ ]:
iJL965_genes = list(i for i in pd.read_csv('./clostridium/paper/iJL965_genes.txt')['id'])
len(iJL965_genes)

In [ ]:
coralme_genes = [i.id.split('RNA_')[1] for i in me.all_genes if 'dummy' not in i.id]

In [ ]:
df = pd.read_csv('./clostridium/building_data/builder_info/complexes_df.txt',index_col=0)

In [ ]:
notinijl = set(coralme_genes) - set(iJL965_genes)
df[df['genes'].str.contains('|'.join([i+'()' for i in notinijl]))]

In [ ]:
notincoral = set(iJL965_genes) - set(coralme_genes)
print(len(notincoral)) 
df[df['genes'].str.contains('|'.join([i+'()' for i in notincoral]))]

Remaining genes that are missing from coralme:
* __CLJU_c42660__ and __CLJU_c42670__ have low-quality annotation from Uniprot, and are missing from BioCyc and GenBank. These genes are not needed for ammonia transport, as CPLX-438 performs the same job.
* __CLJU_c15890__ and __CLJU_c40080__ are deadend proteins in iJL965-ME, performing no job.
* __CLJU_c41100__ is a ribosomal protein in iJL965-ME, but is hypothetical in current annotations.


### Find issue